In [32]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data


In [22]:
df_raw = pd.read_csv('data.csv')
df_raw.head()

df = df_raw.copy()
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     10886 non-null  int64  
 1   dteday      10886 non-null  object 
 2   season      10886 non-null  int64  
 3   yr          10886 non-null  int64  
 4   mnth        10886 non-null  int64  
 5   hr          10886 non-null  int64  
 6   holiday     10886 non-null  int64  
 7   weekday     10886 non-null  int64  
 8   workingday  10886 non-null  int64  
 9   weathersit  10886 non-null  int64  
 10  temp        10886 non-null  float64
 11  atemp       10886 non-null  float64
 12  hum         10886 non-null  float64
 13  windspeed   10886 non-null  float64
 14  casual      10886 non-null  int64  
 15  registered  10886 non-null  int64  
 16  cnt         10886 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 1.4+ MB


In [27]:
df.drop(['instant', 'dteday'], axis=1, inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   yr          10886 non-null  int64  
 2   mnth        10886 non-null  int64  
 3   hr          10886 non-null  int64  
 4   holiday     10886 non-null  int64  
 5   weekday     10886 non-null  int64  
 6   workingday  10886 non-null  int64  
 7   weathersit  10886 non-null  int64  
 8   temp        10886 non-null  float64
 9   atemp       10886 non-null  float64
 10  hum         10886 non-null  float64
 11  windspeed   10886 non-null  float64
 12  casual      10886 non-null  int64  
 13  registered  10886 non-null  int64  
 14  cnt         10886 non-null  int64  
dtypes: float64(4), int64(11)
memory usage: 1.2 MB


In [41]:
class ScooterClassifier(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden, dtype=torch.float64)
        self.act_fn = nn.ReLU()
        self.linear2 = nn.Linear(num_hidden, num_hidden, dtype=torch.float64)
        self.linear3 = nn.Linear(num_hidden, num_hidden, dtype=torch.float64)
        self.linear4 = nn.Linear(num_hidden, num_outputs, dtype=torch.float64)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        x = self.act_fn(x)
        x = self.linear3(x)
        x = self.act_fn(x)
        x = self.linear4(x)
        return x

In [42]:
from sklearn.model_selection import train_test_split

train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)
train, val = train_test_split(train, test_size=0.2, random_state=200)
print(f"Train size: {train.shape}")
print(f"Validation size: {val.shape}")
print(f"Test size: {test.shape}")

Train size: (6967, 15)
Validation size: (1742, 15)
Test size: (2177, 15)


In [74]:
X_train = train.values[:,:-3]
X_test = torch.from_numpy(test.values[:,:-3]).cuda()
X_val = torch.from_numpy(val.values[:,:-3]).cuda()

y_val = torch.from_numpy(val.values[:,-1]).cuda()
y_test = torch.from_numpy(test.values[:,-1]).cuda()

train_dataset = data.TensorDataset(torch.from_numpy(X_train),torch.from_numpy(train.values[:,-1]))

In [75]:
model = ScooterClassifier(num_inputs=12, num_hidden=512, num_outputs=1).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_module = nn.MSELoss(reduction="mean")
train_data_loader = data.DataLoader(train_dataset, batch_size=256, shuffle=True)


In [78]:
def evaluate(model, X, y):
    model.eval()
    with torch.no_grad():
        preds = model(X)
        preds = preds.squeeze(dim=1)

        loss = F.mse_loss(preds, y)

    model.train()
    return loss

In [82]:
import torch.nn.functional as F

model.train()
device = torch.device("cuda")
# Training loop
for epoch in range(500):
    true_preds, num_preds = 0., 0.
    for data_inputs, data_labels in train_data_loader:
        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        data_inputs = data_inputs.to(device, dtype=torch.float64)
        data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(data_inputs)
        preds = preds.squeeze(dim=1)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, data_labels)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

    print(f"Epoch: {epoch}, train_loss: {np.sqrt(loss.item()):.3}, vsl_loss: {np.sqrt(evaluate(model, X_val, y_val).item())}")



Epoch: 0, train_loss: 83.7, vsl_loss: 103.75082916442803
Epoch: 1, train_loss: 1.03e+02, vsl_loss: 102.82872242878973
Epoch: 2, train_loss: 1.01e+02, vsl_loss: 101.35360815818244
Epoch: 3, train_loss: 93.7, vsl_loss: 106.55641969793862
Epoch: 4, train_loss: 1.05e+02, vsl_loss: 100.76040260829899
Epoch: 5, train_loss: 93.1, vsl_loss: 98.31826888655661
Epoch: 6, train_loss: 1.12e+02, vsl_loss: 96.61308318797191
Epoch: 7, train_loss: 1.09e+02, vsl_loss: 96.5018038342987
Epoch: 8, train_loss: 80.6, vsl_loss: 96.09811100396198
Epoch: 9, train_loss: 94.3, vsl_loss: 108.05760574574929
Epoch: 10, train_loss: 83.2, vsl_loss: 97.01442649008743
Epoch: 11, train_loss: 94.5, vsl_loss: 96.15328570672935
Epoch: 12, train_loss: 80.0, vsl_loss: 91.667338605255
Epoch: 13, train_loss: 75.1, vsl_loss: 91.64549351288953
Epoch: 14, train_loss: 1.13e+02, vsl_loss: 88.86254693985204
Epoch: 15, train_loss: 66.3, vsl_loss: 102.37405463076982
Epoch: 16, train_loss: 79.6, vsl_loss: 88.07638621432623
Epoch: 17, tr

## Evaluation data

In [84]:
evaluation_df_raw = pd.read_csv('evaluation_data.csv')
evaluation_df_raw.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-01-20,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881
1,2011-01-20,1,0,1,1,0,4,1,1,0.26,0.2727,0.56,0.0000
2,2011-01-20,1,0,1,2,0,4,1,1,0.26,0.2727,0.56,0.0000
3,2011-01-20,1,0,1,3,0,4,1,1,0.26,0.2576,0.56,0.1642
4,2011-01-20,1,0,1,4,0,4,1,1,0.26,0.2576,0.56,0.1642


In [88]:
evaluation_df = evaluation_df_raw.copy()
evaluation_df.drop(['dteday'], axis=1, inplace=True)
evaluation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      6493 non-null   int64  
 1   yr          6493 non-null   int64  
 2   mnth        6493 non-null   int64  
 3   hr          6493 non-null   int64  
 4   holiday     6493 non-null   int64  
 5   weekday     6493 non-null   int64  
 6   workingday  6493 non-null   int64  
 7   weathersit  6493 non-null   int64  
 8   temp        6493 non-null   float64
 9   atemp       6493 non-null   float64
 10  hum         6493 non-null   float64
 11  windspeed   6493 non-null   float64
dtypes: float64(4), int64(8)
memory usage: 608.8 KB


In [93]:
X = torch.from_numpy(evaluation_df.values).cuda()

In [99]:
with torch.no_grad():
    preds = model(X)
    preds = preds.squeeze(dim=1)

In [95]:
preds[:10]

tensor([  7.5814,   3.1401,   1.6336,   1.9837,   2.3719,   6.7590,  35.4545,
         83.8046, 213.9973,  94.7091], device='cuda:0', dtype=torch.float64,
       grad_fn=<SliceBackward0>)

In [100]:
np_preds = preds.cpu().numpy()

In [101]:
df = pd.DataFrame(np_preds)

In [102]:
df.to_csv('środa_Celiński_Łukasiewicz_predykcje.csv', index=False)